<a href="https://colab.research.google.com/github/Maluxita/notebooks_dataframes/blob/master/evaluate_and_adjust_analysis_and_computations_%5BV_1_2_6%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SECCIÓN [Analysis empty cells]

Esta sección nos ayuda a identificar las columnas que tienen celdas con NaN("Not a Number" para nuestro caso igual a:  "vacías"), así como el tipo de dato presente en cada columna.

> * Recibe una lista con los nombres de las hojas que deseamos leer en cada libro.
> * También recibe una cadena que representa un patrón, el cuál busca los archivos excel cargados.
> * A veces solo nos interesa ver la info de las hojas que tienen celdas NaN, entonces hay que marcar la casilla filter_only_sheets_nan.
> * Marca la casilla print_list_sheets si quieres ver el contenido de cada hoja procesada.

In [37]:
#@title Summary NaN types
sheets =  ['Election','Polling_places']#@param {type:"raw"}
content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
filter_only_sheets_nan = False #@param {type:"boolean"}
view_list_sheets = True #@param {type:"boolean"}


import pandas as pd
from IPython.display import display, HTML

content_books=!ls {content_books}

def count_nan_in_columns_by_sheet(content_books, sheets):
  '''Return dicc with conteo_NaN info by sheet in content_books ''' 
  list_data=[]
  dicc_data={}

  for sheet in sheets:
    for file_name in content_books:
      try:
        sheet_to_df = pd.read_excel(file_name, sheet_name=sheet)
      except:
        print(f'\n ¡AVISO! No existe la hoja {sheet} en este archivo {file_name}')
        continue

      shape = sheet_to_df.shape
      df_types = pd.DataFrame([sheet_to_df.dtypes], columns=sheet_to_df.columns)
      df_types = df_types.append(sheet_to_df, ignore_index=True)
      df_types = df_types.rename(index = {0: 'data_column_type'})
      
      dicc_data = {'Archivo': file_name}
      dicc_data.update({'Hoja': sheet})
      dicc_data.update({'Contenido': str(shape[0]) + ' filas, '+ str(shape[1])+ ' columnas'})

      if sheet_to_df.isnull().sum().sum() == 0:
          msg = 'No tiene ninguna celda tipo NaN'
      else:
          null_columns = sheet_to_df.columns[sheet_to_df.isnull().any()]
          msg=''.join(sheet_to_df[null_columns].isnull().sum().to_string()).split('\n')
          msg=', '.join(msg)

      dicc_data.update({'Aviso': msg })
      list_data.append(dicc_data)

      if view_list_sheets:

          print('\nArchivo:',file_name)
          print('________                                                    ')
          print('\t| Hoja:',sheet, '\t',shape[0], 'filas, ', shape[1], 'columnas')
          print('\t| Aviso conteo NaN:',msg,' |\n')
          if 'Polling_places' in sheet:
            display(HTML(df_types.head(5).to_html()))
          else:
            display(HTML(df_types.to_html()))



  return list_data


list_data=count_nan_in_columns_by_sheet(content_books, sheets)

df=pd.DataFrame([[fila['Archivo'], 
                  fila['Aviso'], 
                  fila['Contenido'], 
                  fila['Hoja']] 
                 for fila in list_data] , columns=['archivo', 'conteo_NaN', 'contenido', 'hoja'])

if filter_only_sheets_nan:
  df=df[~df.conteo_NaN.str.contains('No tiene')]

print('\nRESUMEN:\n')
display(HTML(df.to_html()))


Archivo: sinaloa_municipios_2013_polling_place.xlsx
________                                                    
	| Hoja: Election 	 1 filas,  7 columnas
	| Aviso conteo NaN: No tiene ninguna celda tipo NaN  |



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
data_column_type,int64,int64,object,int64,int64,int64,int64
1,25,2013,Municipios,913380,1962971,0,3



Archivo: sinaloa_municipios_2016_polling_place.xlsx
________                                                    
	| Hoja: Election 	 1 filas,  7 columnas
	| Aviso conteo NaN: No tiene ninguna celda tipo NaN  |



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
data_column_type,int64,int64,object,int64,int64,int64,int64
1,25,2016,Municipios,999355,2064508,0,3



 ¡AVISO! No existe la hoja Election en este archivo sinaloa_municipios_2018_polling_place.xlsx

Archivo: sinaloa_municipios_2013_polling_place.xlsx
________                                                    
	| Hoja: Polling_places 	 4544 filas,  12 columnas
	| Aviso conteo NaN: No tiene ninguna celda tipo NaN  |



,section_id,municipality_id,local_district_id,state_id,"PAN,PRD,PT","PRI,PANAL,PVEM",PASI,MC,"PRI,PANAL,PVEM,PASI","PAN,PRD,PT,PASI",Not_reg,Null_votes
data_column_type,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
1,1726,7,0,25,141,163,12,0,0,0,0,6
2,1726,7,0,25,137,169,11,0,0,0,0,7
3,1727,7,0,25,82,176,29,0,0,0,0,6
4,1727,7,0,25,125,148,21,0,0,0,0,5



Archivo: sinaloa_municipios_2016_polling_place.xlsx
________                                                    
	| Hoja: Polling_places 	 4651 filas,  29 columnas
	| Aviso conteo NaN: No tiene ninguna celda tipo NaN  |



,section_id,local_district_id,municipality_id,state_id,PAN,PRI,PRD,PT,MC,PANAL,PASI,MORENA,PVEM,PES,"PRI,PANAL","MC,PASI",C. Independiente 1,C. Independiente 2,C. Independiente 3,C. Independiente 4,C. Independiente 5,C. Independiente 6,C. Independiente 7,C. Independiente 8,C. Independiente 9,C. Independiente 10,C. Independiente 11,Not_reg,Null_votes
data_column_type,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
1,1726,0,7,25,199,101,12,1,7,8,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9
2,1726,0,7,25,191,104,12,1,4,11,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
3,1727,0,7,25,123,101,16,0,13,16,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
4,1727,0,7,25,132,124,4,1,13,8,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7



 ¡AVISO! No existe la hoja Polling_places en este archivo sinaloa_municipios_2018_polling_place.xlsx

RESUMEN:



,archivo,conteo_NaN,contenido,hoja
0,sinaloa_municipios_2013_polling_place.xlsx,No tiene ninguna celda tipo NaN,"1 filas, 7 columnas",Election
1,sinaloa_municipios_2016_polling_place.xlsx,No tiene ninguna celda tipo NaN,"1 filas, 7 columnas",Election
2,sinaloa_municipios_2013_polling_place.xlsx,No tiene ninguna celda tipo NaN,"4544 filas, 12 columnas",Polling_places
3,sinaloa_municipios_2016_polling_place.xlsx,No tiene ninguna celda tipo NaN,"4651 filas, 29 columnas",Polling_places


## SECCIÓN [Split Computations]

Esta sección nos ayuda a separar una columna apilada dada (C. Independiente n), en múltiples columnas agrupando por una columna pivote (municipality_id)

> * Agregar el nombre de la hoja a leer,
> * Agregar el nombre de la nueva hoja a crear
> * Agregar el patrón de los archivos a leer.




In [ ]:
#@title Functions [Split Computations]
import pandas as pd
from IPython.display import display, HTML
from openpyxl import load_workbook


def some_column_begin_with(columns, phrase='C. Independiente'):
  for column in columns:
    if phrase in column:
      return True
  return False

def filter_columns(columns, phrase='C. Independiente'):
    return filter(lambda c: phrase in c, columns)

def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()


def split_column_in_columns(content_books,sheet, sort_column):
  '''split a stacked column
  (Independent C. n's), in multiple columns
  by filter value of another column (municipality_id)'''

  new_computes=[]
  computes_mun=[]
  phrase='C. Independiente' 
  pivot_column='municipality_id'
  report_sheet_created={}
  list_report_sheet_created=[]

  for file_name in content_books:
    new_computes=[]
    try:
      my_data = pd.read_excel(file_name, sheet_name=sheet)
    except:
      print(f'No existe Polling places en el archivo {file_name}')
      continue
    
    my_data = my_data.sort_values(sort_column)

    if some_column_begin_with(my_data.columns, phrase):
      muns = my_data[pivot_column].unique()

      #print(muns)
      for mun in muns:
        df_mun = my_data.loc[my_data[pivot_column] == mun].copy()

        for n_independiente in filter_columns(my_data.columns, phrase):
          if df_mun[n_independiente].sum() > 0:
            df_mun[n_independiente +'_mun_'+str(mun)] = df_mun[n_independiente]
            
        computes_mun = df_mun.to_dict('records')
        new_computes.extend(computes_mun)
         
      new_data = pd.DataFrame(new_computes)
      new_data = new_data.fillna(0)
      new_data = new_data.astype(int)

      print(f'Se creó la hoja: {new_sheet} en el archivo:{file_name}')
      add_new_sheet(file_name, new_data, new_sheet)
      sheet_created=True
      report_sheet_created={
          'file_name': file_name,
          'sheet_created': sheet_created,
          'new_data_columns': new_data.columns.values,
          'new_data_shape': new_data.shape}

    else:
      print(f'No existen C. Independientes en el archivo: {file_name},solo se ordenó por {sort_column}')
      add_new_sheet(file_name, my_data, sheet)
      sheet_created=False
      report_sheet_created={
          'file_name': file_name,
          'sheet_created': sheet_created,
          'new_data_columns': my_data.columns.values,
          'new_data_shape': my_data.shape}
    list_report_sheet_created.append(report_sheet_created)
    computes_mun={}
  return list_report_sheet_created


In [ ]:
#@title Split columns from computations

sheet="Polling_places"#@param {type:"string"}
new_sheet="Polling_places_separar"#@param {type:"string"}
sort_column="municipality_id"#@param {type:"string"}
_sort_column=True#@param {type:"boolean"}


content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}

content_books=!ls {content_books}


list_report_sheet_created=split_column_in_columns(content_books,sheet, sort_column)


df=pd.DataFrame([[fila['file_name'].split('.')[0], 
                  fila['sheet_created'], 
                  fila['new_data_shape'], 
                  fila['new_data_columns']] 
                 for fila in list_report_sheet_created] , columns=['archivo', 'hoja Nueva?', 'filas,columnas', 'name_columns'])


print('\nRESUMEN:\n')
display(HTML(df.to_html()))


Se creó la hoja: Polling_places_separar en el archivo:quintana-roo_municipios_2018_polling_place.xlsx
No existen C. Independientes en el archivo: sinaloa_municipios_2013_polling_place.xlsx,solo se ordenó por municipality_id
Se creó la hoja: Polling_places_separar en el archivo:sinaloa_municipios_2016_polling_place.xlsx
No existe Polling places en el archivo sinaloa_municipios_2018_polling_place.xlsx

RESUMEN:



,archivo,hoja Nueva?,"filas,columnas",name_columns
0,quintana-roo_municipios_2018_polling_place,True,"(2098, 34)","[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, PANAL, MORENA, PES, PAN,PRD, PAN,MC, PRD,MC, PAN,PRD,MC, PRI,PVEM, PRI,PANAL, PVEM,PANAL, PRI,PVEM,PANAL, MORENA,PT, C. Independiente 0, C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, Not_reg, Null_votes, C. Independiente 0_mun_1, C. Independiente 1_mun_2, C. Independiente 2_mun_3, C. Independiente 3_mun_7, C. Independiente 4_mun_10]"
1,sinaloa_municipios_2013_polling_place,False,"(4544, 12)","[section_id, municipality_id, local_district_id, state_id, PAN,PRD,PT, PRI,PANAL,PVEM, PASI, MC, PRI,PANAL,PVEM,PASI, PAN,PRD,PT,PASI, Not_reg, Null_votes]"
2,sinaloa_municipios_2016_polling_place,True,"(4651, 40)","[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PT, MC, PANAL, PASI, MORENA, PVEM, PES, PRI,PANAL, MC,PASI, C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, C. Independiente 5, C. Independiente 6, C. Independiente 7, C. Independiente 8, C. Independiente 9, C. Independiente 10, C. Independiente 11, Not_reg, Null_votes, C. Independiente 2_mun_1, C. Independiente 3_mun_1, C. Independiente 11_mun_4, C. Independiente 1_mun_10, C. Independiente 4_mun_11, C. Independiente 5_mun_11, C. Independiente 6_mun_11, C. Independiente 8_mun_12, C. Independiente 9_mun_12, C. Independiente 10_mun_12, C. Independiente 7_mun_15]"


## SECCIÓN 1: DEFINIR BIBLIOTECAS Y ARCHIVOS

In [ ]:
#@title Importamos bibliotecas
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [ ]:
#@title Definir funciones a usar
def add_new_sheet_index(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists new sheet with other index will be created'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  #print(writer.sheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def types_columns_unique(types_dataframe):
  ''' generate a set of types columns from columns dataframe'''
  return ','.join(map(str, set(list(types_dataframe))))


def adjust_integers(content_books, sheet_name, new_sheet_name, regex_pattern_aditional=r'^-*$|^o+$|^O+$'):
  ''' Try to convert all columns in a sheet_name to integer, 
      from each book in content_books '''
  all_columns_integer={}
  print(f'\nSe creará NUEVA HOJA "{new_sheet_name}" para cada uno de los archivos siguientes, con los ajustes que se muestran en summary')
  for book in content_books:
    sustituir_vacios = True
    original_all_numbers = False
    print('\n\nLeyendo Libro:',book)
    
    try:
      df_temp = pd.read_excel(book,sheet_name=sheet_name)
    except:
      print(f'\tNo tiene la hoja: {sheet_name}')
      continue

    all_columns_integer[book]={}

    try:
      df_temp = df_temp.astype(int, errors='raise')
      sustituir_vacios = False
      original_all_numbers = True
    except ValueError as e:
      print('\n\tAVISO: Se encontraron valores que no se pueden convertir a entero directamente...')
      #print('\n\t\t!Info error 1:',e)
      all_columns_integer[book]['integer'] = False
    if sustituir_vacios:
      try:
        df_temp=df_temp.replace(r'^\s*$', 0, regex=True).replace(regex_pattern_aditional, 0, regex=True)
        #print(f'\n\tTRATAREMOS de SUSTITUIR valores VACÍOS, y los que cumplan este PATRÓN: {regex_pattern_aditional} por ceros')
      except ValueError as e:
        print('\n\tAVISO: Algún valor NO SE PUDO SUSTITUIR por 0\'s \:\( ')
        #print ('\n\t\t!Info error 2:',e)
    df_temp=df_temp.fillna(0)

    try:
      df_temp = df_temp.astype(int, errors='raise')
      all_columns_integer[book]['integer'] = True
    except ValueError as e:
      print('\n\tAVISO: Existe algún valor que no se puede convertir a entero :S')
      #print('\n\t\t!Info error 3', e)
      all_columns_integer[book]['integer'] = False
    
    all_columns_integer[book]['original_all_numbers'] = original_all_numbers
    all_columns_integer[book]['types_unique'] = types_columns_unique(df_temp.dtypes)

    g = dict(df_temp.columns.to_series().groupby(df_temp.dtypes).groups)   
    types_columns={}

    for k,v in g.items():
      if not 'int' in (str(k)):
        types_columns[str(k)]=','.join(list(v))

    if not all_columns_integer[book]['integer']:
      print('\n\tAVISO: Intenta un proceso individual para cada columna ...')
    types_columns.update({'book':book.split('.')[0]})
    all_columns_integer[book]['types_by_column'] = types_columns
    add_new_sheet(name_book=book,df=df_temp,name_sheet=new_sheet_name)


  print('\nFin del proceso :)')
  return all_columns_integer


In [ ]:
#@title Leemos cuales son los archivos con terminación xlsx
content_books = !ls *.xlsx | tr '\n' '\n'

## SECCIÓN 2: CONVERSIÓN ENTEROS 
######  

*   Modifica a integer los valores de las columnas
*   De no ser posible la conversión, intenta reemplazar a 0's espacios en blanco y los que cumplan con un patrón regular.
*   En el mismo archivo agrega una hoja nueva 'Polling_places_corrections'

In [ ]:
#@title Column to Integer Conversion Proccess
all_columns_integer = adjust_integers(content_books = content_books, 
                                    sheet_name = 'Polling_places', 
                                    new_sheet_name = 'Polling_places_corrections',
                                    regex_pattern_aditional = r'^-*$|^o+$|^O+$|^/+$')



Se creará NUEVA HOJA "Polling_places_corrections" para cada uno de los archivos siguientes, con los ajustes que se muestran en summary


Leyendo Libro: sinaloa_municipios_2013_polling_place.xlsx


Leyendo Libro: sinaloa_municipios_2016_polling_place.xlsx


Leyendo Libro: sinaloa_municipios_2018_polling_place.xlsx
	No tiene la hoja: Polling_places

Fin del proceso :)


In [ ]:
#@title Summary Conversion Integers
df = pd.DataFrame([[key.split('.')[0],value['integer'],value['original_all_numbers'], value['types_unique']] for key,value in all_columns_integer.items()], columns=['name_book', 'Finally Integer?','Began with numbers?', 'Types unique'])
df

,name_book,Finally Integer?,Began with numbers?,Types unique
0,sinaloa_municipios_2013_polling_place,True,True,int64
1,sinaloa_municipios_2016_polling_place,True,True,int64


### SECCIÓN 2.1 Suggest changes in columns cells

In [ ]:
#@title Summary Columns cannot convert to Integers in a book
columnas_not_int=pd.DataFrame([v['types_by_column'] for v in all_columns_integer.values() if not v['integer']])
pd.set_option('display.max_colwidth', 0)
columnas_not_int


""


In [ ]:
#@title Proccess to suggest changes in columns
def fix_multiple_types(n): 
    import re
    tipo=str(type(n))
    x=n
    if 'int' in tipo:
      return n
    if 'str' in tipo:
      numbers_patron = re.compile(r"[\d]", re.X)
      number=''.join(numbers_patron.findall(n))
      try: 
        n=int(number)
        print( '\t\t\tAhora: ',str(type(n)), str(n), '\tAntes:', tipo, str(x))
        return n
      except:
        pass
    elif 'float' in tipo:
      if n >= 0:
        n=int(n)
        return n
    print('\t\t\tAhora:', str(type(0)),str(0), '\tAntes:', str(type(x)), str(x))
    return 0

def fix_multiple_types_suggest_columns(n): 
    import re
    tipo=str(type(n))
    x=n
    if 'int' in tipo:
      return n
    if 'str' in tipo:
      numbers_patron = re.compile(r"[\d]", re.X)
      number=''.join(numbers_patron.findall(n))
      try: 
        n=int(number)
        n=''.join(['Ahora: ',str(type(n)),' ', str(n), ' Antes: ', tipo, str(x)])
        return n
      except:
        pass
    elif 'float' in tipo:
      if n >= 0:
        n=int(n)
        return n
    return ''.join(['Ahora: ', str(type(0)),' ', str(0), ' Antes: ', str(type(x)), str(x)])
   

for k,v in all_columns_integer.items():
  df_temp_columns_suggest = pd.read_excel(k,sheet_name='Polling_places')
  df_temp_sheet_suggest = pd.read_excel(k,sheet_name='Polling_places')

  print('LIBRO:',k,'\t')
 
  if not v['integer']:
    for key, tipos in v['types_by_column'].items():
      if key == 'book':
        continue
      print('TIPO:',key)
      
      for c in tipos.split(','):
        print('\n\tColumna:\n\t',c,'\n')
        df_temp_sheet_suggest[c]=list(map(fix_multiple_types,list(df_temp_sheet_suggest[c])))
        #df_temp_columns_suggest[c+'suggested']= df_temp_columns_suggest[c].map(fix_multiple_types_suggest_columns)
        df_temp_columns_suggest[c]= df_temp_columns_suggest[c].map(fix_multiple_types_suggest_columns)

    df_temp_sheet_suggest.astype(int, errors='ignore')
    df_temp_columns_suggest.astype(int, errors='ignore')
    

    add_new_sheet(name_book=k,df=df_temp_sheet_suggest,name_sheet='Polling_places_corrections')
    add_new_sheet(name_book=k,df=df_temp_columns_suggest,name_sheet='Polling_places_suggested')


LIBRO: aguascalientes_diputados_2013_polling_place.xlsx 	
LIBRO: baja-california_municipios_2013_polling_place (4).xlsx 	
LIBRO: baja-california_municipios_2016_polling_place.xlsx 	


## SECCIÓN 3: REVISAR HOJA ELECCION

In [ ]:
#@title Comprobar hoja Election
columnas_plantilla_election1=['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity']
#columnas_plantilla_Polling_places=['section_id', 'local_disctrict_id', 'municipality_id', 'state_id','Not_reg','Null_votes']
columnas_plantilla_election=set(['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity'])
lista_elections=[]
dicc_election={}

from IPython.display import display, HTML
import pandas as pd

content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
content_books =  !ls {content_books}

for k in content_books:
  print(k)
  try:
    xls = pd.ExcelFile(k)
  except:
    print('No es un archivo válido')
    continue
  if  not 'Election' in xls.sheet_names:
    print(f'\n\tNo existe la hoja Election para este archivo {k}')
    continue
  try:
    data_election=pd.read_excel(k,sheet_name='Election')
  except:
    print(f'No se puede leer el archivo{k}')
    continue
  dicc_election['book']=k
  columnas_hoja_election=set(data_election.columns)

  if  not columnas_plantilla_election-columnas_hoja_election:
    print('\tColumnas igual a la plantilla!\n')
    header_equal=True
  else:
    print('Falta al menos una columna con respecto a la plantilla\n')
    header_equal=False

  dicc_election['header_equal']=header_equal

  dicc_election['types_column']=list(data_election.dtypes)
  dicc_election['values_fila']=list(data_election.iloc[0])
  lista_elections.append(dicc_election)
  dicc_election['total_values']=len(data_election.iloc[0])
  dicc_election['total_header']=len(data_election.iloc[0])

  df_=pd.DataFrame([dicc_election['types_column'],dicc_election['values_fila'] ], index=['types','values'],columns=columnas_plantilla_election1)
  
  display(HTML(df_.to_html()))
  print()





sinaloa_municipios_2013_polling_place.xlsx
	Columnas igual a la plantilla!



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
types,int64,int64,object,int64,int64,int64,int64
values,25,2013,Municipios,913380,1962971,0,3



sinaloa_municipios_2016_polling_place.xlsx
	Columnas igual a la plantilla!



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
types,int64,int64,object,int64,int64,int64,int64
values,25,2016,Municipios,999355,2064508,0,3



sinaloa_municipios_2018_polling_place.xlsx
No es un archivo válido


## SECCIÓN 4: EVALUACIÓN CONTINUIDAD COLUMNA

In [ ]:
#@title functions [evaluación continuidad]
import pandas as pd
def continuidad_columna(content_books=[], 
                        sheet_name='Polling_places_corrections',
                        column_to_evaluate='local_district_id'
                        ):
  ''' Evaluá si existe continuidad en columma determinada de una hoja del libro dada, compromando que sea vacía la diferencia simétrica'''
  evaluacion_distritos={}

  #print ('\nPara todos los archivos siguientes se evaluará la hoja:',sheet_name,'y la columna:' ,column_to_evaluate)
  for book in content_books:

    #print('\nLibro:',book)
    try:
      df_temp = pd.read_excel(book,sheet_name=sheet_name)
    except:
      print(f'\tNo existe la hoja {sheet_name} en el archivo: {book}')
      continue

    evaluacion_distritos[book]={}


    _df=df_temp.agg({column_to_evaluate:['max','min']})
    _unicos = df_temp[column_to_evaluate].unique()

    evaluacion_distritos[book]['min'] = _df[column_to_evaluate]['min']
    evaluacion_distritos[book]['max'] = _df[column_to_evaluate]['max']
    evaluacion_distritos[book]['shape']= df_temp.shape
    evaluacion_distritos[book]['size']= df_temp.size

    a = set(_unicos)
    b = set(np.arange(evaluacion_distritos[book]['min'],evaluacion_distritos[book]['max'] + 1))
    c = a^b

    evaluacion_distritos[book]['diferencia']=", ".join(map(str,c))
  return evaluacion_distritos

In [ ]:
#@title Process to evaluate continuity in a column
import numpy as np
from IPython.display import display, HTML


content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
content_books =  !ls {content_books}
column_to_evaluate =  "municipality_id"#@param {type:"string"}
sheet_name="Polling_places_separar"#@param {type:"string"}

evaluacion_distritos=continuidad_columna(content_books=content_books, 
                        sheet_name=sheet_name,
                        column_to_evaluate=column_to_evaluate
                        )

df = pd.DataFrame([[key.split('.')[0],value['diferencia'],value['min'], value['max'], value['shape'], value['size']] for key,value in evaluacion_distritos.items()], 
                  columns=['Libro', 'IDs No Continuos', 'Mínimo', 'Máximo', 'Filas,Columnas', 'Total Celdas'])
display(HTML(df.to_html()))


	No existe la hoja Polling_places_separar en el archivo: sinaloa_municipios_2013_polling_place.xlsx
	No existe la hoja Polling_places_separar en el archivo: sinaloa_municipios_2018_polling_place.xlsx


,Libro,IDs No Continuos,Mínimo,Máximo,"Filas,Columnas",Total Celdas
0,sinaloa_municipios_2016_polling_place,,1,18,"(6749, 54)",364446


## SECCIÓN 5: EDICIÓN DE NOMBRES DE COLUMNAS

In [ ]:
#@title Definir funciones a usar
def reemplazar_nombres_columnas(df_temp, reemplazar):
  '''Recibe un nuevo nombre de columna y lo reemplaza por una anteriormente dado'''
  
  if reemplazar == 'Y':

    print("\t¿Cuál es nombre de columna a reemplazar? ", end="")
    columna_anterior = input()
    print("\t¿Cuál es el nuevo nombre de columna?", end="")
    columna_nueva = input()
    print(f"\n\tSe reemplazará: {columna_anterior} por {columna_nueva}")

    if columna_anterior in df_temp.columns and columna_nueva:
  
      df_temp.rename(columns={ columna_anterior :columna_nueva},inplace=True)
      
      print ('\n\tREEMPLAZO LISTO!\n')
      print('\n\tESTAS SON LAS COLUMNAS AHORA:\n\n\t',df_temp.columns)
    else:
      print(f'{columna_anterior} No es una columna existente! y/o recuerda agregar el texto por el cual sustituir')

  print('\n\t ¿Deseas reemplazar el nombre de otra columna? (Y/N)')
  reemplazar = input()

  if reemplazar == 'Y':
    df_temp=reemplazar_nombres_columnas(df_temp, reemplazar)
    print('\tSe actualizará el nombre de la columna en el archivo de excel')
  else:
    print('\tFinalizando edición de columnas de este archivo...')

  return df_temp


def rename_columns(content_books=[], sheet_name='Polling_places_corrections'):
  '''Reads the name and renames certain columns of a sheet'''
  for book in content_books:

    print('\n\nNOMBRE DEL ARCHIVO:',book)
    try:
      df_temp=pd.read_excel(book,sheet_name=sheet_name)
    except:
      print(f'No existe la hoja {sheet_name} en el archivo: {book}')
      continue
    print('\n\n\tESTAS SON LAS COLUMNAS PRESENTES:\n\n')
    print('\t',df_temp.columns)
    print('\n\t¿Deseas reemplazar el nombre de alguna columna? (Y/N)')

    reemplazar = input()

    if reemplazar == 'Y': 
      df_temp = reemplazar_nombres_columnas(df_temp,reemplazar)
      add_new_sheet(name_book=book,df=df_temp,name_sheet=sheet_name)
      print('\tSe actualizó el nombre de la columna en el archivo de excel')
    else:
      if reemplazar == 'N':
        print('\tBye!')
      else:
        print('\tOpción no válida, pasaremos al siguiente archivo...')

  return print('\nProceso finalizado :D')

In [ ]:
#@title Proccess to help us to rename columns

from IPython.display import display, HTML


content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
content_books =  !ls {content_books}
sheet_name="Polling_places_separar"#@param {type:"string"}


rename_columns(content_books=content_books, sheet_name=sheet_name)



NOMBRE DEL ARCHIVO: sinaloa_municipios_2013_polling_place.xlsx
No existe la hoja Polling_places_separar en el archivo: sinaloa_municipios_2013_polling_place.xlsx


NOMBRE DEL ARCHIVO: sinaloa_municipios_2016_polling_place.xlsx


	ESTAS SON LAS COLUMNAS PRESENTES:


	 Index(['section_id', 'local_district_id', 'municipality_id', 'state_id', 'PAN',
       'PRI', 'PRD', 'PVEM', 'PT', 'MC', 'PANAL', 'MORENA', 'PES', 'PAN,PRD',
       'PAN,MC', 'PRD,MC', 'PAN,PRD,MC', 'PRI,PVEM', 'PRI,PANAL', 'PVEM,PANAL',
       'PRI,PVEM,PANAL', 'MORENA,PT', 'C. Independiente 0',
       'C. Independiente 1', 'C. Independiente 2', 'C. Independiente 3',
       'C. Independiente 4', 'Not_reg', 'Null_votes',
       'C. Independiente 0_mun_1', 'C. Independiente 1_mun_2',
       'C. Independiente 2_mun_3', 'C. Independiente 3_mun_7',
       'C. Independiente 4_mun_10', 'PASI', 'MC,PASI', 'C. Independiente 5',
       'C. Independiente 6', 'C. Independiente 7', 'C. Independiente 8',
       'C. Independiente 9', 

## SECCIÓN 6: Descargar archivos

In [ ]:
archivos= ! ls *e.xlsx | tr '\n' '\n'
from google.colab import files

for archivo in archivos:
  files.download(archivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>